# Baseline model

In [1]:
import numpy as np
import pandas as pd

### Opening data sets:

In [2]:
recipes = pd.read_csv("../raw_data/recipes.csv")

In [3]:
recipes.head()

,id,name,ingredients,ingredient_ids
0,137739,arriba baked winter squash mexican style,"['winter squash', 'mexican seasoning', 'mixed ...","[7933, 4694, 4795, 3723, 840, 5006, 6270]"
1,31490,a bit different breakfast pizza,"['prepared pizza crust', 'sausage patty', 'egg...","[5481, 6324, 2499, 4717, 6276, 1170]"
2,59389,alouette potatoes,"['spreadable cheese with garlic and herbs', 'n...","[1170, 4918, 6426, 5185, 7099, 5006, 6009, 627..."
3,5289,apple a day milk shake,"['milk', 'vanilla ice cream', 'frozen apple ju...","[4717, 7474, 2946, 150]"
4,25274,aww marinated olives,"['fennel seeds', 'green olives', 'ripe olives'...","[2587, 3437, 5002, 3184, 5324, 5068, 5058, 131..."


In [4]:
sample = recipes.head(100)
sample

,id,name,ingredients,ingredient_ids
0,137739,arriba baked winter squash mexican style,"['winter squash', 'mexican seasoning', 'mixed ...","[7933, 4694, 4795, 3723, 840, 5006, 6270]"
1,31490,a bit different breakfast pizza,"['prepared pizza crust', 'sausage patty', 'egg...","[5481, 6324, 2499, 4717, 6276, 1170]"
2,59389,alouette potatoes,"['spreadable cheese with garlic and herbs', 'n...","[1170, 4918, 6426, 5185, 7099, 5006, 6009, 627..."
3,5289,apple a day milk shake,"['milk', 'vanilla ice cream', 'frozen apple ju...","[4717, 7474, 2946, 150]"
4,25274,aww marinated olives,"['fennel seeds', 'green olives', 'ripe olives'...","[2587, 3437, 5002, 3184, 5324, 5068, 5058, 131..."
...,...,...,...,...
95,190,20 000 prize winning chili con carne,"['lean chuck', 'lean pork', 'onion', 'garlic c...","[4187, 4206, 5010, 3203, 821, 7233, 7655, 1329..."
96,50662,25 000 casserole,"['beef', 'flour', 'paprika', 'onion', 'cream o...","[422, 2683, 5168, 5010, 1917, 332, 6270, 5667,..."
97,177187,250 chocolate chip cookies recipe,"['butter', 'brown sugar', 'vanilla', 'blended ...","[840, 800, 7449, 632, 335, 954, 4958, 2683, 69..."
98,118843,250 00 chocolate chip cookies,"['butter', 'white sugar', 'brown sugar', 'eggs...","[840, 7790, 800, 2499, 7449, 2683, 4975, 6270,..."


In [5]:
# has all ingredients, incluing the repeated ones
ingredients = pd.read_csv("../raw_data/ingredient_keys.csv")[['id','replaced','count']].drop_duplicates()

In [6]:
ingredients.head()

,id,replaced,count
0,4308,lettuce,4507
42,2744,french vanilla pudding and pie filling mix,5
43,6843,stove top stuffing mix,114
45,1910,cream cheese,9915
86,1168,cheddar,13304


In [7]:
# drop duplicates
ingredients_list = ingredients['id'].to_list()

# Buildind the base df "ingredient_pairs"

In [8]:
ingredient_pairs = pd.DataFrame(columns = list(ingredients['id'].sort_values()),index=list(ingredients['id'].sort_values()), data=0)

In [9]:
ingredient_pairs

,0,1,2,3,4,5,6,7,8,9,...,8013,8014,8015,8016,8017,8018,8019,8020,8021,8022
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8019,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8021,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Creating loop to fill df with pair combinations

In [10]:
import ast
import time

In [11]:
recipes_list = recipes['ingredient_ids'].to_numpy()
recipes_list = np.array([ast.literal_eval(item) for item in recipes_list])

<ipython-input-11-c812c18def8b>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  recipes_list = np.array([ast.literal_eval(item) for item in recipes_list])


In [12]:
#### descomentar se quiser executar de novo a criação da combinação dos ingredientes
# %%time
# for ingr_id in range(8023):
#     print(f'looking for recipes containing ingredient {ingr_id}', end='\r')
#     for idx, recipe_ingredients in enumerate(recipes_list):
#         if ingr_id in recipe_ingredients:
#             ingredient_pairs.loc[ingr_id,recipe_ingredients] += 1
#             recipe_ingredients.remove(ingr_id)
#             if not recipe_ingredients:
#                 np.delete(recipes_list, idx, 0)

In [13]:
#### descomentar se quiser criar o csv
# ingredient_pairs.to_csv('../raw_data/ingredient_pairs.csv', index=False)

In [14]:
ingredient_pairs_csv = pd.read_csv('../raw_data/ingredient_pairs.csv')
ingredient_pairs_csv

,0,1,2,3,4,5,6,7,8,9,...,8013,8014,8015,8016,8017,8018,8019,8020,8021,8022
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,13,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,29,0,0,1,0
8019,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,187,0,8,0
8020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8021,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3661,0


# Creating vector thats counts single ingredient occurences

In [15]:
# vector = ingredient_pairs[]
vector = np.diagonal(ingredient_pairs_csv)

In [16]:
vector

array([   1,   13,    6, ...,    1, 3661,    2])

# Dividing the pairs array by the single ingredient vector

In [17]:
ingredient_pairs_csv = np.array(ingredient_pairs_csv)

In [18]:
combination_percentage = ingredient_pairs_csv / vector

<ipython-input-18-adafd556ccaa>:1: RuntimeWarning: invalid value encountered in true_divide
  combination_percentage = ingredient_pairs_csv / vector


In [19]:
combination_percentage = pd.DataFrame(combination_percentage)

In [20]:
combination_percentage

,0,1,2,3,4,5,6,7,8,9,...,8013,8014,8015,8016,8017,8018,8019,8020,8021,8022
0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,1.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8018,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000273,0.0
8019,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.002185,0.0
8020,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0
8021,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0


In [21]:
#### descomentar se quiser criar o csv
# combination_percentage.to_csv('../raw_data/combination_percentage.csv', index=False)

# Get the ingredient row to output results

In [22]:
# fazer a linha de valores do ingrediente:
# percentage.iloc[0:2,1:2].T
pd.concat([combination_percentage.iloc[0:2,1:2].T,combination_percentage.iloc[1:2,2:8022]], axis=1)

,0,1,2,3,4,5,6,7,8,9,...,8012,8013,8014,8015,8016,8017,8018,8019,8020,8021
1,0.0,1.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
